<a href="https://colab.research.google.com/github/hshuai97/Colab20210803/blob/main/5_datasets_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

注意：使用Yao等人的数据集: [link](https://github.com/yao8839836/text_gcn)，并加以整理；其中，20ng使用的是cleaned之后的数据（corpus的数据数量不对），其他使用的是corpus中的原始数据

In [ ]:
import os
import re
import time
import random

import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split  # for train and dev set split

#  Split train and test set
根据训练集和测试集的映射文件将训练数据train和测试数据test分开

In [ ]:

def clean_text(NAME, text):
    re_url = re.compile(r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    re_email = re.compile('(?:[a-z0-9!#$%&\'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&\'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])')
    stop_words = stopwords.words('english')
    # for 20ng
    '''
    text = re.sub(r'(From:\s+[^\n]+\n)', '', text)
    #text = re.sub(r'(Subject:[^\n]+\n)', '', text)
    text = re.sub(r'(Organization:[^\n]+\n)', '', text)
    text = re.sub(r'(Distribution:[^\n]+\n)', '', text)
    text = re.sub(r'(Lines:[^\n]+\n)', '', text)
    text = re.sub(r'(([\sA-Za-z0-9\-]+)?[A|a]rchive-name:[^\n]+\n)', '', text)
    text = re.sub(r'(Last-modified:[^\n]+\n)', '', text)
    text = re.sub(r'(Version:[^\n]+\n)', '', text)
    text = re.sub(r'(NNTP-Posting-Host:[^\n]+\n)', '', text)
    text = re.sub(r'(Nntp-Posting-Host:[^\n]+\n)', '', text)
    text = re.sub(r'(In article[^\n]+\n)', '', text)
    text = re.sub(r'(In [^\n]+writes:\n)', '', text)
    text = re.sub(r'(X-Newsreader:[^\n]+\n)', '', text)
    text = re.sub(r'(In-Reply-To: [^\n]+\n)', '', text)
    text = re.sub(r'(Article-I.D.: [^\n]+\n)', '', text)
    text = re.sub(r'(Expires: [^\n]+\n)', '', text)
    '''
    
    # for 5 datasets
    text = text.lower()
    text = text.strip()
    text = re.sub(re_url, '', text)
    text = re.sub(re_email, '', text)

    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)

    text = re.sub(r"[^A-Za-z().?!\'\`]", " ", text)
    text = re.sub(r"\(", "", text)
    text = re.sub(r"\)", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r'`', '', text)

    text = re.sub(r"\s{2,}", " ", text)  # Remove [ \t\n\r\f\v].

    # clean stopwords
    if NAME in ['mr']:
      return text
    else:
      text = [t for t in word_tokenize(text) if t not in stop_words]
      text = ' '.join(text)

      return text


def Clean_and_Save(path, dataset_name):
  NAME = dataset_name
  if NAME not in ['20ng', 'r8', 'r52', 'oh', 'mr']:
    raise ValueError('The dataset is not support')
  raw_x = []
  with open(os.path.join(path, NAME+'.txt'), encoding='latin1') as f:
    data = f.readlines()
    print(f'Total sample: {len(data)}')

  with open(os.path.join(path, NAME+'_mapping.txt'), encoding='latin1') as f:
    map = f.readlines()
    print(f'Total_mapping: {len(map)}')

  if len(map) != len(data):
    raise Exception('map size not equal data')
  
  # clean sample and save
  tes_x, tes_y = [], []
  tra_x, tra_y = [], []
  for i in range(len(map)):
    clea = clean_text(NAME, data[i]).strip()
    labe = map[i].split('\t')

    if labe[len(labe)-2] in ['20news-bydate-test', 'test']:
      tes_x.append(clea)
      tes_y.append(re.sub(r'\n', '', labe[len(labe)-1]))
    
    elif labe[len(labe)-2] in ['20news-bydate-train', 'train', 'training']:
      tra_x.append(clea)
      tra_y.append(re.sub(r'\n', '', labe[len(labe)-1]))
    
  if len(tes_y)!=len(tes_x) or len(tra_x)!=len(tra_y):
    raise Exception('numbef of x is not equal y')

  tra_res = list(zip(tra_y, tra_x))
  random.shuffle(tra_res)
  tra_y, tra_x = zip(*tra_res)

  tes_res = list(zip(tes_y, tes_x))
  random.shuffle(tes_res)
  tes_y, tes_x = zip(*tes_res)

  df_tra = pd.DataFrame({'tra_y': tra_y, 'tra_x': tra_x})
  df_tes = pd.DataFrame({'tes_y': tes_y, 'tes_x': tes_x})

  df_tra.columns = range(df_tra.shape[1])  # Remove the header
  df_tes.columns = range(df_tes.shape[1])  # Remove the header


  df_tra.to_csv(f'/content/drive/MyDrive/Colab_Notebooks/TextLevelGNN/data/{NAME}-stemmed.txt', index=False, sep='\t',header=None)
  df_tes.to_csv(f'/content/drive/MyDrive/Colab_Notebooks/TextLevelGNN/data/{NAME}-test-stemmed.txt', index=False, sep='\t',header=None)


path = '/content/drive/MyDrive/Colab_Notebooks/TextLevelGNN/data/'

t0 = time.time()
for name in ['20ng', 'r8', 'r52', 'oh', 'mr']:
  Clean_and_Save(path, name)
  print(f'Time: {time.time() - t0:.4f}s')
  print('='*50)


Total sample: 18846
Total_mapping: 18846
Time: 45.2457s
Total sample: 7674
Total_mapping: 7674
Time: 55.0459s
Total sample: 9100
Total_mapping: 9100
Time: 66.2946s
Total sample: 7400
Total_mapping: 7400
Time: 86.5907s
Total sample: 10662
Total_mapping: 10662
Time: 91.1435s


# Split train and dev set

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 400})'''))

def DataInfo_Divide(path, dataset_name):  # Statistic dataset information
  NAME = dataset_name

  y_stem = []
  with open(os.path.join(path, NAME+'-stemmed.txt'), encoding='latin1') as f:
    all = f.readlines()
    for line in all:
        t = line.split('\t')
        y_stem.append(t[0])
  
  # Statistic
  target_names = list(set(y_stem))
  print(f'Number of categories is {len(target_names)} in {NAME}-stemmed.txt')

  # Split
  if NAME in ['r52']:  # r52 some label only has 1
    train, dev = train_test_split(all, test_size=0.1, shuffle=True, random_state=42)
  else:
    train, dev = train_test_split(all, test_size=0.1, shuffle=True, random_state=42, stratify=y_stem)

  # Write new train and dev set
  with open(os.path.join(path, NAME+'-train-stemmed.txt'), encoding='latin1', mode='w') as f:
    f.write(''.join(train))
  
  with open(os.path.join(path, NAME+'-dev-stemmed.txt'), encoding='latin1', mode='w') as f:
    f.write(''.join(dev))
  


  # Check
  y_tra = []
  with open(os.path.join(path, NAME+'-train-stemmed.txt'), encoding='latin1') as f:
      tra = f.readlines()
      for line in tra:
        t = line.split('\t')
        y_tra.append(t[0])
  
  tar_nam = list(set(y_tra))  # Target names
  index = pd.Index(y_tra)
  print(f'Number of categories is {len(tar_nam)} in {NAME}-train-stemmed.txt,\n{index.value_counts()}')


  y_dev = []
  with open(os.path.join(path, NAME+'-dev-stemmed.txt'), encoding='latin1') as f:
      de = f.readlines()
      for line in de:
        t = line.split('\t')
        y_dev.append(t[0])
  tar_nam_ = list(set(y_dev))  # target names
  print(f'Number of categories is {len(tar_nam_)} in {NAME}-dev-stemmed.txt')
  print(f'\nNew total train: {len(tra)+len(de)}, new train: {len(tra)}, new dev: {len(de)}')


path = '/content/drive/MyDrive/Colab_Notebooks/TextLevelGNN/data/'

t1 = time.time()
for name in ['20ng', 'r8', 'r52', 'oh', 'mr']:
  DataInfo_Divide(path, name)
  print(f'Time: {time.time() - t1:.4f}s')
  print('='*50)

<IPython.core.display.Javascript object>

Number of categories is 20 in 20ng-stemmed.txt
Number of categories is 20 in 20ng-train-stemmed.txt,
rec.sport.hockey            540
soc.religion.christian      539
rec.motorcycles             538
rec.sport.baseball          537
sci.crypt                   535
rec.autos                   535
sci.med                     535
comp.windows.x              534
sci.space                   534
sci.electronics             532
comp.os.ms-windows.misc     532
comp.sys.ibm.pc.hardware    531
comp.graphics               526
misc.forsale                526
comp.sys.mac.hardware       520
talk.politics.mideast       508
talk.politics.guns          491
alt.atheism                 432
talk.politics.misc          418
talk.religion.misc          339
dtype: int64
Number of categories is 20 in 20ng-dev-stemmed.txt

New total train: 11314, new train: 10182, new dev: 1132
Time: 1.0253s
Number of categories is 8 in r8-stemmed.txt
Number of categories is 8 in r8-train-stemmed.txt,
earn        2556
acq         

In [ ]:
text = 'wow, you are so smart.'

#text = re.sub(r',', '.', text)
res = sent_tokenize(text)
res

['wow, you are so smart.']